#### Quick notes from tesnorboard video
* https://www.youtube.com/watch?v=eBbEDRsCmv4&feature=youtu.be&t=97

```
# tf.summary.FileWriter(n):
# classthat writes data for tensorboard..

writer = tf.summary.FileWriter("/tmp/blah/1")
writer.add_graph(sess.graph)
```
* And run ... 
```
tensorboard --logdir /tmp/mnist_demo/1
```


In [1]:
# boiler plate
from collections import Counter
import pickle
from importlib import reload
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler


import ipdb
import mytf.utils as mu

In [2]:
print(tf.executing_eagerly())


False


In [3]:
with open('models/2019-07-21T1815-UTC-outdata-SUBSET50k.pkl', 'rb') as fd:
    julydata = pickle.load(fd)

In [5]:
training_indices = np.arange(0, julydata['x_train'].shape[0], 1)
training_indices.shape


(50000,)

In [6]:

class_weights = {0: 1.0, 1: 1.3, 2: 1.0, 3: 1.0}
training_indices = np.arange(0, julydata['x_train'].shape[0], 1)
print('training indices...', training_indices.shape)

dataset_batches = mu.build_dataset_weighty_v3(
    julydata,
    training_indices, 
    class_weights,
    batch_size=1000)

training indices... (50000,)
Start build v3: .. doesnt add up to 1.0
num slices 50
size_remainder,  0
Counter({0: 519, 2: 416, 3: 51, 1: 14})
weights_per_class,  Tensor("truediv:0", shape=(4,), dtype=float32)
1004.1999999999998
Counter({0: 514, 2: 408, 3: 54, 1: 24})
weights_per_class,  Tensor("truediv_1:0", shape=(4,), dtype=float32)
1007.1999999999995
Counter({0: 528, 2: 396, 3: 54, 1: 22})
weights_per_class,  Tensor("truediv_2:0", shape=(4,), dtype=float32)
1006.5999999999997
Counter({0: 519, 2: 417, 3: 48, 1: 16})
weights_per_class,  Tensor("truediv_3:0", shape=(4,), dtype=float32)
1004.8
Counter({0: 527, 2: 408, 3: 53, 1: 12})
weights_per_class,  Tensor("truediv_4:0", shape=(4,), dtype=float32)
1003.5999999999997
Counter({0: 527, 2: 410, 3: 48, 1: 15})
weights_per_class,  Tensor("truediv_5:0", shape=(4,), dtype=float32)
1004.4999999999997
Counter({0: 542, 2: 381, 3: 62, 1: 15})
weights_per_class,  Tensor("truediv_6:0", shape=(4,), dtype=float32)
1004.4999999999997
Counter({0: 503,

In [7]:
with tf.Session() as sess:

    # okay... now try use that ..
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64,   dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=(None, 256, 3), 
                  ),
        # 4 because 'A', 'B', 'C', 'D'.
        tf.keras.layers.Dense(4)
    ])

    %time loss_history = mu.do_train(model, dataset_batches)

    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [8]:
tf.enable_eager_execution()


ValueError: tf.enable_eager_execution must be called at program startup.

#### Grr, so that kind of sucks that I got this error ...

```
RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.
```

I guess this means the dataset tooling I was using cannot be used 
for both eager and non-eager exeuction. 

Looking at the code from the youtube tutorial, https://github.com/martinwicke/tf-dev-summit-tensorboard-tutorial/blob/master/mnist.py
batches are iterated using this `next_batch(100)`

```python
for i in range(2001):
    batch = mnist.train.next_batch(100)
```

But I tried that on my dataset and the next_batch function was not implemented 
for the dataset that I built using 

```python
    dataset = tf.data.Dataset.from_tensor_slices(
        (train_tensor, label_tensor, weights_tensor))
```
